## __Chapter 06: Untitled__
---
by Daniel Weller & Holger Mann

In [ ]:
data.packages <- c("bigmemory", "data.table", "rgdal", "rgeos")
lapply(data.packages, require, character.only=T); rm(data.packages)

stat.packages <- c("bivariate", "extRemes", "ks", "mev", "mvtnorm")
lapply(stat.packages, require, character.only=T); rm(stat.packages)

if(.Platform$OS.type!="windows"){
  paths <- c("/Volumes/Climate Rasters/", "/Volumes/Analysis Storage", "~/GitHub/Report/")} else {
    paths <- c("D:/Climate Rasters/", "E:/Analysis Storage/", "C:/Users/Cloud/GitHub/Report/")}
setwd(paths[3]); options(scipen=10); set.seed(12345); setDTthreads(16)
rasterOptions(progress="text", maxmemory=1e+09, chunksize=1e+08)

In [ ]:
cold.sens <- big.matrix(nrow=cells, ncol=length(seq(0, 1, by=1e-2)), init=NULL, separated=FALSE,
                        backingpath=".", backingfile="Cold_Sens_Back", descriptorfile="Cold_Sens_Desc")

for(j in 1979:1990){
  for(i in cell.list){
    values <- fread(paste(paths[2], "Cold Wave Large/Cell ", i, ".csv",sep=""))
    values <- values[Year %in% j:(j+29)][, Rise := round(Thresh, digits=2) - MaxDay][WaveID >= 1]
    values[, `:=`(Excess=sum(round(Rise, digits=2)), Length=max(DaysCount, na.rm=TRUE)), by=.(WaveID)]
    values <- unique(values[, .(WaveID, Excess, Length)])[Excess >= 0.03 & Length >= 3]
    values <- values[, .(Count=.N), by=.(Excess, Length)]
    if(!exists("rest.wave")){rest.wave <- values} else {
      rest.wave <- merge(rest.wave, values, all=TRUE, by=c("Excess", "Length"), suffixes=c("_1", "_2"))
      rest.wave[, Count := sum(Count_1, Count_2, na.rm=TRUE), by=1:nrow(rest.wave)]
      rest.wave <- rest.wave[, .(Excess, Length, Count)]}
    if(i %in% c(seq(1, cells, 1e+3), cells)){
      if(!exists("cold.wave")){cold.wave <- rest.wave; rm(rest.wave)} else {
        cold.wave <- merge(cold.wave, rest.wave, all=TRUE, by=c("Excess", "Length"), suffixes=c("_1", "_2"))
        cold.wave[, Count := sum(Count_1, Count_2, na.rm=TRUE), by=1:nrow(cold.wave)]
        cold.wave <- cold.wave[, .(Excess, Length, Count)]; rm(rest.wave)}}
    cat(paste("Phase 1 - Current Cell:", i, "\n", seq="")); rm(values)}
  fwrite(cold.wave, paste("./Cold_Wave_Model_", j,"_", j+29, ".csv", sep=""))
  
  wght <- as.numeric(cold.wave$Count)
  mins <- c(min(cold.wave[,1]), min(cold.wave[,2]))
  maxs <- c(max(cold.wave[,1]), max(cold.wave[,2]))
  cold.kde <- kde(cold.wave[,1:2], w=wght, xmin=mins, xmax=maxs, gridsize=maxs)
  cold.sim <- rkde(1e+5, cold.kde); cold.fun <- ebvcdf(cold.sim[,1], cold.sim[,2])
  save(cold.fun, file=paste(".Cold_Wave_Kernel_", j,"_", j+29, ".RData", sep=""))
  
  for(i in cell.list){
    values <- fread(paste(paths[2], "Cold Wave Large/Cell ", i, ".csv",sep=""))
    values <- values[Year %in% j:(j+29)][, Rise := MaxDay - round(Thresh, digits=2)][WaveID >= 1]
    values[, `:=` (Excess=sum(round(Rise, digits=2)), Length=max(DaysCount, na.rm=TRUE)), by=.(WaveID)]
    values <- unique(values[, .(WaveID, Excess, Length)])[Excess >= 0.03 & Length >= 3]
    for(k in 1:nrow(values)){values[k, Prob := cold.fun(values[k,2], values[k,3])]}
    for(l in 1:length(seq(0, 1, by=1e-2))){cold.sens[i,l] <- sum(values[,4] >= seq(0, 1, by=1e-2)[l])}
    cat(paste("Phase 2 - Current Cell:", i, "in", j, "\n", seq="")); rm(values)}
  write.big.matrix(cold.sens, paste("./Cold_Wave_Scale_", j,"_", j+29, ".csv", sep="")); cold.sens <- NA}


heat.sens <- big.matrix(nrow=cells, ncol=length(seq(0, 1, by=1e-2)), init=NULL, separated=FALSE,
                        backingpath=".", backingfile="Heat_Sens_Back", descriptorfile="Heat_Sens_Desc")

for(j in 1979:1990){
  for(i in cell.list){
    values <- fread(paste(paths[2], "Heat Wave Large/Cell ", i, ".csv",sep=""))
    values <- values[Year %in% j:(j+29)][, Rise := MaxDay - round(Thresh, digits=2)][WaveID >= 1]
    values[, `:=`(Excess=sum(round(Rise, digits=2)), Length=max(DaysCount, na.rm=TRUE)), by=.(WaveID)]
    values <- unique(values[, .(WaveID, Excess, Length)])[Excess >= 0.03 & Length >= 3]
    values <- values[, .(Count=.N), by=.(Excess, Length)]
    if(!exists("rest.wave")){rest.wave <- values} else {
      rest.wave <- merge(rest.wave, values, all=TRUE, by=c("Excess", "Length"), suffixes=c("_1", "_2"))
      rest.wave[, Count := sum(Count_1, Count_2, na.rm=TRUE), by=1:nrow(rest.wave)]
      rest.wave <- rest.wave[, .(Excess, Length, Count)]}
    if(i %in% c(seq(1, cells, 1e+3), cells)){
      if(!exists("heat.wave")){heat.wave <- rest.wave; rm(rest.wave)} else {
        heat.wave <- merge(heat.wave, rest.wave, all=TRUE, by=c("Excess", "Length"), suffixes=c("_1", "_2"))
        heat.wave[, Count := sum(Count_1, Count_2, na.rm=TRUE), by=1:nrow(heat.wave)]
        heat.wave <- heat.wave[, .(Excess, Length, Count)]; rm(rest.wave)}}
    cat(paste("Phase 1 - Current Cell:", i, "\n", seq="")); rm(values)}
  fwrite(heat.wave, paste("./Heat_Wave_Model_", j,"_", j+29, ".csv", sep=""))

  wght <- as.numeric(heat.wave$Count)
  mins <- c(min(heat.wave[,1]), min(heat.wave[,2]))
  maxs <- c(max(heat.wave[,1]), max(heat.wave[,2]))
  heat.kde <- kde(heat.wave[,1:2], w=wght, xmin=mins, xmax=maxs, gridsize=maxs)
  heat.sim <- rkde(1e+5, heat.kde); heat.fun <- ebvcdf(heat.sim[,1], heat.sim[,2])
  save(heat.fun, file=paste("./Heat_Wave_Kernel_", j,"_", j+29, ".RData", sep=""))

  for(i in cell.list){
    values <- fread(paste(paths[2], "Heat Wave Large/Cell ", i, ".csv",sep=""))
    values <- values[Year %in% j:(j+29)][, Rise := MaxDay - round(Thresh, digits=2)][WaveID >= 1]
    values[, `:=` (Excess=sum(round(Rise, digits=2)), Length=max(DaysCount, na.rm=TRUE)), by=.(WaveID)]
    values <- unique(values[, .(WaveID, Excess, Length)])[Excess >= 0.03 & Length >= 3]
    for(k in 1:nrow(values)){values[k, Prob := heat.fun(values[k,2], values[k,3])]}
    for(l in 1:length(seq(0, 1, by=1e-2))){heat.sens[i,l] <- sum(values[,4] >= seq(0, 1, by=1e-2)[l])}
    cat(paste("Phase 2 - Current Cell:", i, "in", j, "\n", seq="")); rm(values)}
  write.big.matrix(heat.sens, paste("./Heat_Wave_Scale_", j,"_", j+29, ".csv", sep="")); heat.sens <- NA}